In [38]:
import torch
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os
import re

In [7]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()

        # Define the neural network layers
        self.fc1 = nn.Linear(10, 16)  # First fully connected layer with 10 input features and 16 output features
        self.fc2 = nn.Linear(16, 12)  # Second fully connected layer with 16 input features and 12 output features
        self.fc3 = nn.Linear(12, 6)   # Third fully connected layer with 12 input features and 6 output features

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))  # Apply ReLU activation after the first layer
        x = nn.ReLU()(self.fc2(x))  # Apply ReLU activation after the second layer
        x = self.fc3(x)             # No activation for the last layer
        return x

# Instantiate the model
model = SimpleNN()

In [109]:
batch_size = 14 # You can change this value to your preferred batch size
input_data = torch.randn(batch_size, 10) 

In [110]:
input_data.shape, model(input_data).shape

(torch.Size([14, 10]), torch.Size([14, 6]))

In [111]:
my_folder = 'x'
idx = 11
#for idx, tensor in enumerate(dataloader0):
torch.save(input_data, f"{my_folder}/tensor{idx}.pt")

In [113]:

def store_generated_data(gen_model,folder_name,n): #might need to pass in input shape?
    """
    gen_model is the teacher
    folder_name is just where to put it
    n is how many samples.
    """
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    inputs_file_path = os.path.join(folder_name, "inputs")
    targets_file_path = os.path.join(folder_name, "targets")

    if not os.path.exists(inputs_file_path):
        os.makedirs(inputs_file_path)
    if not os.path.exists(targets_file_path):
        os.makedirs(targets_file_path)

    ##get the max file number from each file.

    files = os.listdir(inputs_file_path)
    pattern = re.compile(r'input_(\d+)')
    integers = [int(match.group(1)) for file in files if (match := pattern.match(file))]
    
    if integers:
        idx_last = max(integers) + 1
    else:
        idx_last = 1
    
    gen_model.eval()

    for idx in range(n): #tqdm this?
        #generate data, this will be a proper scheme
        input_data = torch.randn(1, 10) #this is a batch_size of 1.  i will have to mess with llama to do this right. 

        #pass through model
        target_data = gen_model(input_data)
        torch.save(input_data, f"{inputs_file_path}/input_{idx_last + idx}.pt")
        torch.save(target_data, f"{targets_file_path}/target_{idx_last + idx}.pt")
    print(f'generate datapoints from {idx_last} to {idx_last + n - 1}')

In [114]:
n=20 #number of training points to make. 
folder_name = "my_folder_1"
gen_model = model
store_generated_data(gen_model,folder_name,n)

generate datapoints from 81 to 100


In [124]:
class FolderDataset(Dataset):
    #assumes you used the above store_generated_data
    def __init__(self, folder):
        self.files = os.listdir(f"{folder}/inputs")
        self.folder = folder

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # Load the tensor
        input_tensor = torch.load(f"{self.folder}/inputs/{self.files[idx]}")
        
        idx_match = re.search(r'input_(\d+)', self.files[idx])
        if idx_match:
            target_idx = int(match.group(1))
            target_tensor = torch.load(f"{self.folder}/targets/target_{target_idx}.pt")
        else:
            raise ValueError(f"Invalid filename format: {self.files[idx]}")
        
        
        return input_tensor.squeeze(), target_tensor.squeeze()

In [128]:

dataset = FolderDataset(folder_name)
dataloader = DataLoader(dataset, batch_size=7, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [129]:
num_epochs = 5
for epoch in range(num_epochs):
    for data in dataloader:
        # Here, 'data' is whatever is saved in the torch files in your folder.
        # Assuming it's a tensor you want to use as input:
        inputs, targets = data
        print(inputs.shape, targets.shape)
        outputs = model(inputs)
        print(outputs.shape)
        loss = torch.nn.functional.mse_loss(outputs, targets)  # Mock loss for testing
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Loss: {loss.item()}")

torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.0019948857370764017
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.0030165761709213257
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.000832923746202141
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.0016743320738896728
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.0017386144027113914
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.0013637410011142492
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.0014175515389069915
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.0011508219176903367
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.0014936315128579736
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.0020261285826563835
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6])
Loss: 0.0012276447378098965
torch.Size([7, 10]) torch.Size([7, 6])
torch.Size([7, 6